# Prepare mutual dataset

In [43]:
from datasets import load_dataset
import json 

ds = load_dataset("EleutherAI/mutual","mutual")

/home/iker/miniconda3/envs/transformers/lib/python3.11/site-packages/datasets/load.py:1461: FutureWarning: The repository for EleutherAI/mutual contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/EleutherAI/mutual
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [64]:
ds["validation"][2]

{'answers': 'C',
 'options': ["f : please wait a second . i 'll get dr. smith for you . he is right in his office .",
  "f : sorry . we do n't know where dr. smith is . could you please try later ?",
  'f : ok. dr. smith is at his home now . you can try tomorrow .',
  "f : ok. dr. smith is in his office now . i 'll let him know you called when he gets home ."],
 'article': "m : hello , is this doctor , smith 's office ? f : yes , it is . may i help you ? m : yes , i 'd like to speak to doctor smith , please ? f : doctor smith went home this afternoon . may i ask who is calling ? m : this is jim white .",
 'id': 'dev_100'}

In [63]:
ds["validation"][3]

{'answers': 'D',
 'options': ['m : the home phone number is 77221069 , right ?',
  "m : dr. smith 's office phone number is 77231059 , right ?",
  'm : the home phone number is 77231058 , right ?',
  "m : dr. smith 's home phone number is 77231059 , right ?"],
 'article': "m : hello , is this doctor , smith 's office ? f : yes , it is . may i help you ? m : yes , i 'd like to speak to doctor smith , please ? f : doctor smith went home this afternoon . may i ask who is calling ? m : this is jim white . f : oh yes , mister white . doctor smith asked me to give you his home phone number . m : just a moment , please . yes , what 's the number ? f : his number is 77231059 .",
 'id': 'dev_101'}

In [58]:
def letter_position(letter):
    letter = letter.lower()
    position = ord(letter) - ord('a')
    return position

def extract_turns(conversation):
    # Split the conversation by the pattern " f :" and " m :" to separate each turn
    turns = conversation.split("f : ")
    final_turns = []

    # First part might start with m:, so handle that
    if turns[0].startswith("m :"):
        sub_turns = turns[0].split("m : ")
        for sub_turn in sub_turns:
            if sub_turn.strip():  # Ensure it's not an empty string
                final_turns.append({"role":"A", "content": sub_turn.strip()})
        turns = turns[1:]

    # Process the remaining turns
    for turn in turns:
        sub_turns = turn.split(" m : ")
        if sub_turns[0].strip():  # Ensure it's not an empty string
            final_turns.append({"role":"B", "content": sub_turns[0].strip()})
        if len(sub_turns) > 1 and sub_turns[1].strip():
            final_turns.append({"role":"A", "content": sub_turns[1].strip()})
    
    return final_turns



def preprocess_example(example):
    options = example["options"]
    options = [extract_turns(option) for option in options]
    conversation = extract_turns(example["article"])
    correct_option =letter_position(example["answers"])
    return {"conversation": conversation, "candidates": options, "label": correct_option}
    


In [59]:
print(json.dumps(preprocess_example(ds["train"][1]),ensure_ascii=False, indent=4))

{
    "conversation": [
        {
            "role": "A",
            "content": "is everything ready for billy 's birthday party ?"
        },
        {
            "role": "B",
            "content": "yes . i finished making the birthday cake and i put everything on the table . did you find the party hats ?"
        },
        {
            "role": "A",
            "content": "yes , i did . i put one for each child on the table . i put up the big `` happy birthday '' sign , too ."
        },
        {
            "role": "B",
            "content": "thanks , honey . do you think we have enough for the kids to eat and drink ?"
        },
        {
            "role": "A",
            "content": "i 'm sure we do . here 's enough food to feed an army ."
        },
        {
            "role": "B",
            "content": "that birthday cake looks beautiful , but you have n't put any candles on it yet ."
        }
    ],
    "candidates": [
        [
            {
                "role"

In [60]:
# Print the dataset to a jsonl file
for split in ["train", "validation"]:
    with open(f"mutual_{split}.jsonl", "w") as f:
        for example in ds[split]:
            processed_example = preprocess_example(example)
            print(json.dumps(processed_example, ensure_ascii=False), file=f)


# Daily Dialog

In [66]:
from datasets import load_dataset
import json 
import random 

ds = load_dataset("li2017dailydialog/daily_dialog")

/home/iker/miniconda3/envs/transformers/lib/python3.11/site-packages/datasets/load.py:1461: FutureWarning: The repository for li2017dailydialog/daily_dialog contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/li2017dailydialog/daily_dialog
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/11118 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [70]:
ds["train"][0]

{'dialog': ['Say , Jim , how about going for a few beers after dinner ? ',
  ' You know that is tempting but is really not good for our fitness . ',
  ' What do you mean ? It will help us to relax . ',
  " Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? ",
  " I guess you are right.But what shall we do ? I don't feel like sitting at home . ",
  ' I suggest a walk over to the gym where we can play singsong and meet some of our friends . ',
  " That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them . ",
  ' Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too . ',
  " Good.Let ' s go now . ",
  ' All right . '],
 'act': [3, 4, 2, 2, 2, 3, 4, 1, 3, 4],
 'emotion': [0, 0, 0, 0, 0, 0, 4, 4, 4, 4]}

In [125]:
def preprocess_example(example):
    conversation = example["dialog"]
    for i in range(len(conversation)):
        conversation[i] = {"role": "A" if i % 2 == 0 else "B", "content": conversation[i].strip()}
    return conversation

def generate_candidates(conversation_idx, dataset,num_candidates:int=4, full_negative_prop:float=0.0):
    all_negatives = random.random() < full_negative_prop
    # Pick a random position to cut the conversation
    conversation = dataset[conversation_idx]
    cut_position = random.randint(len(conversation)//2, len(conversation) - 1)
    # Cut the conversation
    correct_next_turn = conversation[cut_position]
    conversation = conversation[:cut_position]
    # Pick three random conversations, but avoid conversation_idx
    other_conversations = [i for i in range(len(dataset)) if i != conversation_idx]
    candidates = random.sample(other_conversations, num_candidates if all_negatives > 0.0 else num_candidates-1)
    candidates = [dataset[candidate] for candidate in candidates]
    # Pick random turns from the candidates
    candidates = [[random.choice(candidate)] for candidate in candidates]
    turn_of_correct_next = correct_next_turn["role"]
    for candidate in candidates:
        candidate[0]["role"] = turn_of_correct_next
    
    if all_negatives > 0.0:
        insert_position = -1
    else:
        insert_position = random.randint(0, num_candidates-1)
        candidates.insert(insert_position, [correct_next_turn])
    return {"conversation": conversation, "candidates": candidates, "label": insert_position}

    

In [126]:
dataset = [preprocess_example(example) for example in ds["train"]]

In [127]:
dataset[0]

[{'role': 'A',
  'content': 'Say , Jim , how about going for a few beers after dinner ?'},
 {'role': 'B',
  'content': 'You know that is tempting but is really not good for our fitness .'},
 {'role': 'A', 'content': 'What do you mean ? It will help us to relax .'},
 {'role': 'B',
  'content': "Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ?"},
 {'role': 'A',
  'content': "I guess you are right.But what shall we do ? I don't feel like sitting at home ."},
 {'role': 'B',
  'content': 'I suggest a walk over to the gym where we can play singsong and meet some of our friends .'},
 {'role': 'A',
  'content': "That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them ."},
 {'role': 'B',
  'content': 'Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too .'},
 {'role': 'A', 'content': "Good.Let ' s go now ."},
 {'role':

In [128]:
generate_candidates(0, dataset,full_negative_prop=0.0)

{'conversation': [{'role': 'A',
   'content': 'Say , Jim , how about going for a few beers after dinner ?'},
  {'role': 'B',
   'content': 'You know that is tempting but is really not good for our fitness .'},
  {'role': 'A', 'content': 'What do you mean ? It will help us to relax .'},
  {'role': 'B',
   'content': "Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ?"},
  {'role': 'A',
   'content': "I guess you are right.But what shall we do ? I don't feel like sitting at home ."},
  {'role': 'B',
   'content': 'I suggest a walk over to the gym where we can play singsong and meet some of our friends .'},
  {'role': 'A',
   'content': "That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them ."}],
 'candidates': [[{'role': 'B',
    'content': "I don't know exactly . I only hear it's financed by a famous film star ."}],
  [{'role': 'B',
    'content': 'Sounds great to me ! If they are

In [129]:
for split in ["train", "validation", "test"]:
    dataset = [preprocess_example(example) for example in ds[split]]
    with open(f"daily_dialog_{split}.jsonl", "w") as f:
        for i in range(len(dataset)):
            processed_example = generate_candidates(i, dataset,full_negative_prop=0.0 if split == "train" else 0.1)
            print(json.dumps(processed_example, ensure_ascii=False), file=f)

In [134]:
d = {"conversation": [{"role": "A", "content": "Good morning , sir . Is there a bank near here ?"}, {"role": "B", "content": "There is one . 5 blocks away from here ?"}, {"role": "A", "content": "Well , that's too far.Can you change some money for me ?"}], "candidates": [[{"role": "B", "content": "Alright . I'm sorry , Miss . According to our file , this prescription has already been refilled twice ."}], [{"role": "B", "content": "Of course , let me help you ."}], [{"role": "B", "content": "Would you mind if I ask some personal questions ?"}], [{"role": "B", "content": "Surely , of course . What kind of currency have you got ?"}]], "label": 3}


In [135]:
print(json.dumps(d,ensure_ascii=False, indent=4))

{
    "conversation": [
        {
            "role": "A",
            "content": "Good morning , sir . Is there a bank near here ?"
        },
        {
            "role": "B",
            "content": "There is one . 5 blocks away from here ?"
        },
        {
            "role": "A",
            "content": "Well , that's too far.Can you change some money for me ?"
        }
    ],
    "candidates": [
        [
            {
                "role": "B",
                "content": "Alright . I'm sorry , Miss . According to our file , this prescription has already been refilled twice ."
            }
        ],
        [
            {
                "role": "B",
                "content": "Of course , let me help you ."
            }
        ],
        [
            {
                "role": "B",
                "content": "Would you mind if I ask some personal questions ?"
            }
        ],
        [
            {
                "role": "B",
                "content": "Sur

4

[1, 2, 3]